In [2]:
import clickhouse_connect
import pandas as pd
import numpy as np
# from arayuzVerileriGetirme import *
pd.set_option('display.max_columns', None)

ModuleNotFoundError: No module named 'clickhouse_connect'

In [4]:
magazaKodu = "1001"
urunKodu = "30000332"
table_name="master_egitim"

In [6]:
client = clickhouse_connect.get_client(
    host='l3flqlcyjf.germanywestcentral.azure.clickhouse.cloud',
    user='default',
    password='e11Uq697dnZq_',
    secure=True
)

print("Result:", client.query("SELECT 1").result_set[0][0])


Result: 1


In [8]:
dfTum = client.query_df("SELECT * FROM master_egitim LIMIT 5")


In [10]:
dfTum.head()

,tarih,hafta_gunu,ay,yil,yil_gunu,season_code,month_position_code,promotion_day,satismiktari,satistutarikdvsiz,stok,degerlenmisstok,satisFiyati,indirimYuzdesi,enflasyon,stok_out,stok_out_gun_sayisi,magazakodu,urunkodu,ailekodu,altailekodu,malgrubukodu,reyonkodu,sektorkodu,bulundugusehirkodu,magaza_metrekare,50 TL OPERASYON,500 TL OPERASYON,Açılış Katalogları,Alkollü Ürünler,Decot,GAZETE ILANI,HYBR,Hybris % Kampanya,Kapanış Mağazaları,KATALOG,LEAFLET,Mağ.İçi Akt-FMCG,Mağ.İçi Akt-GıdaDışı,Mağ.İçi Akt-TazeGıda,VKA0,ZKAE,Tayin edilmedi,promosyonVar,ozelgun,formatstore,ilce,avmcadde,cluster,cografi_bolge,sezon,aktifPromosyonAdi,aktifPromosyonKodu,iconkod,temp,lag_1,lag_2,lag_3,lag_4,lag_7,lag_10,lag_14,lag_21,lag_28,roll_mean_7,roll_mean_14,roll_mean_21,roll_std_7,roll_std_14,roll_std_21,roll_median_7,roll_median_14,roll_median_21,roll_min_7,roll_min_14,roll_min_21,roll_max_7,roll_max_14,roll_max_21
0,2023-01-01,7,1,2023,1,1,1,0,64,221.839996,1339,3128.729980,3.466250,0.0,0.0,0,0,1001,30000264,10410,1041005,104100496,104,1,34,7000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,Hipermarket,ataşehir,avm,Büyük Hiper,Marmara,yerel,Tayin edilmedi,17,1,8.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-02,1,1,2023,2,1,1,0,52,180.250000,1383,3231.600098,3.466346,0.0,0.0,0,0,1001,30000264,10410,1041005,104100496,104,1,34,7000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Hipermarket,ataşehir,avm,Büyük Hiper,Marmara,yerel,Tayin edilmedi,17,1,7.7,64,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-03,2,1,2023,3,1,1,0,24,83.220001,1359,3175.439941,3.467500,0.0,0.0,0,0,1001,30000264,10410,1041005,104100496,104,1,34,7000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Hipermarket,ataşehir,avm,Büyük Hiper,Marmara,yerel,Tayin edilmedi,17,2,8.4,52,64,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-04,3,1,2023,4,1,4,0,33,114.410004,1398,3266.629883,3.466970,0.0,0.0,0,0,1001,30000264,10410,1041005,104100496,104,1,34,7000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Hipermarket,ataşehir,avm,Büyük Hiper,Marmara,yerel,Tayin edilmedi,17,2,9.3,24,52,64,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-05,4,1,2023,5,1,4,0,133,450.670013,1313,3067.689941,3.388496,0.0,0.0,0,0,1001,30000264,10410,1041005,104100496,104,1,34,7000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Hipermarket,ataşehir,avm,Büyük Hiper,Marmara,yerel,Tayin edilmedi,17,2,10.4,33,24,52,64,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
from typing import List, Optional


def get_inventory_store_performance(
    client,
    table_name: str = "master_egitim",
    region_ids: Optional[List[str]] = None,
    store_ids: Optional[List[str]] = None,
    category_ids: Optional[List[str]] = None,
    product_ids: Optional[List[str]] = None,
) -> dict:
    """
    GET /api/inventory/store-performance
    """

    where_clauses = ["1 = 1"]

    if region_ids:
        regions = ", ".join(f"'{r}'" for r in region_ids)
        where_clauses.append(f"cografi_bolge IN ({regions})")

    if store_ids:
        stores = ", ".join(f"'{s}'" for s in store_ids)
        where_clauses.append(f"toString(magazakodu) IN ({stores})")

    if category_ids:
        cats = ", ".join(f"'{c}'" for c in category_ids)
        where_clauses.append(f"toString(sektorkodu) IN ({cats})")

    if product_ids:
        prods = ", ".join(f"'{p}'" for p in product_ids)
        where_clauses.append(f"toString(urunkodu) IN ({prods})")

    where_sql = " AND ".join(where_clauses)

    query = f"""
        SELECT
            toString(magazakodu)                                            AS storeId,
            concat(bulundugusehirkodu, ' - ', ilce)                          AS storeName,

            sum(stok)                                                       AS stockLevel,

            round(
                100 * sum(satismiktari) /
                nullIf(sum(satismiktari) + sum(stok), 0),
                2
            )                                                               AS sellThroughRate,

            round(
                sum(satismiktari) / nullIf(countDistinct(toDate(tarih)), 0),
                0
            )                                                               AS dailySales,

            round(
                sum(stok) /
                nullIf(
                    sum(satismiktari) / nullIf(countDistinct(toDate(tarih)), 0),
                    0
                ),
                0
            )                                                               AS daysOfInventory,

            round(
                (
                    0.4 * (
                        100 * sum(satismiktari) /
                        nullIf(sum(satismiktari) + sum(stok), 0)
                    )
                    +
                    0.3 * (
                        100 - abs(
                            (
                                sum(stok) /
                                nullIf(
                                    sum(satismiktari) /
                                    nullIf(countDistinct(toDate(tarih)), 0),
                                    0
                                )
                            ) - 30
                        )
                    )
                    +
                    0.3 * (
                        sum(satismiktari) /
                        nullIf(countDistinct(toDate(tarih)), 0)
                    )
                ),
                2
            )                                                               AS storeEfficiency

        FROM {table_name}
        WHERE {where_sql}
        GROUP BY magazakodu, bulundugusehirkodu, ilce
        ORDER BY storeEfficiency DESC
    """

    rows = client.query(query).result_rows

    return {
        "stores": [
            {
                "storeId": r[0],
                "storeName": r[1],
                "stockLevel": int(r[2] or 0),
                "sellThroughRate": float(r[3] or 0),
                "dailySales": int(r[4] or 0),
                "daysOfInventory": int(r[5] or 0),
                "storeEfficiency": float(r[6] or 0),
            }
            for r in rows
        ]
    }


In [56]:
from typing import List, Optional
from datetime import datetime


def get_inventory_alerts(
    client,
    table_name: str = "master_egitim",
    region_ids: Optional[List[str]] = None,
    store_ids: Optional[List[str]] = None,
    severity: Optional[str] = None,
    alert_type: Optional[str] = None,
) -> dict:

    where_clauses = ["1 = 1"]

    if region_ids:
        regions = ", ".join(f"'{r}'" for r in region_ids)
        where_clauses.append(f"cografi_bolge IN ({regions})")

    if store_ids:
        stores = ", ".join(f"'{s}'" for s in store_ids)
        where_clauses.append(f"toString(magazakodu) IN ({stores})")

    where_sql = " AND ".join(where_clauses)

    query = f"""
        SELECT
            toString(urunkodu)                              AS sku,
            toString(urunkodu)                              AS productName,
            concat(bulundugusehirkodu, ' - ', ilce)         AS storeName,

            stok                                            AS currentStock,
            minimumstok                                    AS threshold,
            tahminlenen_talep                               AS forecastedDemand,
            satismiktari                                    AS todaysSales,
            maksimumstok                                    AS maxStock,

            tarih
        FROM {table_name}
        WHERE {where_sql}
    """

    rows = client.query(query).result_rows
    alerts = []

    for i, r in enumerate(rows):
        (
            sku,
            product_name,
            store_name,
            current_stock,
            threshold,
            forecast,
            todays_sales,
            max_stock,
            _,
        ) = r

        alert = None

        if current_stock == 0:
            alert = ("stockout", "high", "Stok tükendi. Acil tedarik gerekiyor.", "reorder")

        elif current_stock <= threshold:
            alert = ("reorder", "medium", "Stok kritik seviyenin altına düştü.", "reorder")

        elif current_stock > max_stock:
            alert = ("overstock", "low", "Fazla stok tespit edildi.", "promotion")

        elif todays_sales == 0 and current_stock > 0:
            alert = ("deadstock", "high", "Uzun süredir satılmayan ürün.", "review")

        elif forecast > todays_sales * 2:
            alert = ("surge", "medium", "Talep artışı bekleniyor.", "transfer")

        if not alert:
            continue

        alert_type_val, sev, msg, action = alert

        if severity and sev != severity:
            continue

        if alert_type and alert_type_val != alert_type:
            continue

        alerts.append(
            {
                "id": f"alert-{sku}-{i}",
                "type": alert_type_val,
                "sku": sku,
                "productName": product_name,
                "storeName": store_name,
                "message": msg,
                "date": datetime.now().strftime("%b %d, %H:%M"),
                "severity": sev,
                "metrics": {
                    "currentStock": int(current_stock),
                    "threshold": int(threshold),
                    "forecastedDemand": int(forecast),
                },
                "recommendation": _recommendation(alert_type_val),
                "actionType": action,
            }
        )

    return {"alerts": alerts}


In [58]:
response = get_inventory_alerts(
    client=client,
    table_name="master_egitim",
    region_ids=["Marmara"],
    severity="high",
)

response

DatabaseError: Received ClickHouse exception, code: 47, server response: Code: 47. DB::Exception: Unknown expression identifier `minimumstok` in scope SELECT toString(urunkodu) AS sku, toString(urunkodu) AS productName, concat(bulundugusehirkodu, ' - ', ilce) AS storeName, stok AS currentStock, minimumstok AS threshold, tahminlenen_talep AS forecastedDemand, satismiktari AS todaysSales, maksimumstok AS maxStock, tarih FROM master_egitim WHERE (1 = 1) AND (cografi_bolge IN ('Marmara')). (UNKNOWN_IDENTIFIER) (version 25.8.1.36912 (official build)) (for url https://l3flqlcyjf.germanywestcentral.azure.clickhouse.cloud:8443)

In [40]:
response

{'summary': {'lowGrowth': {'count': 5051217, 'severity': 'medium'},
  'highGrowth': {'count': 5040341, 'severity': 'info'},
  'forecastErrors': {'count': 6539161,
   'criticalCount': 3528128,
   'severity': 'high'},
  'inventory': {'count': 19202763,
   'stockout': 1624425,
   'overstock': 17506789,
   'reorder': 363550,
   'severity': 'high'}},
 'totalAlerts': 35833482}

In [48]:
response

{'categories': [{'value': '1001_1', 'label': '1', 'storeValue': '1001'},
  {'value': '1004_1', 'label': '1', 'storeValue': '1004'},
  {'value': '1012_1', 'label': '1', 'storeValue': '1012'},
  {'value': '1013_1', 'label': '1', 'storeValue': '1013'},
  {'value': '1014_1', 'label': '1', 'storeValue': '1014'},
  {'value': '1016_1', 'label': '1', 'storeValue': '1016'},
  {'value': '1017_1', 'label': '1', 'storeValue': '1017'},
  {'value': '1022_1', 'label': '1', 'storeValue': '1022'},
  {'value': '1024_1', 'label': '1', 'storeValue': '1024'},
  {'value': '1026_1', 'label': '1', 'storeValue': '1026'},
  {'value': '1028_1', 'label': '1', 'storeValue': '1028'},
  {'value': '1051_1', 'label': '1', 'storeValue': '1051'},
  {'value': '1052_1', 'label': '1', 'storeValue': '1052'},
  {'value': '1053_1', 'label': '1', 'storeValue': '1053'},
  {'value': '1054_1', 'label': '1', 'storeValue': '1054'},
  {'value': '1055_1', 'label': '1', 'storeValue': '1055'},
  {'value': '1057_1', 'label': '1', 'store

In [16]:
# yapilmayan
# `GET /api/dashboard/metrics`
### `GET /api/dashboard/revenue-chart`
### `GET /api/dashboard/promotions`
# get_forecast_product_data(client)
#### `GET /api/forecast/product-data`

NameError: name 'yapilmayan' is not defined

In [ ]:
# response = get_regions_hierarchy(client)
# response = get_stores(client)
# response = get_categories(client)
# response = get_products(client)
# response = get_reyonlar(client)
# response = get_historical_chart(client)
# ---
# query = get_alerts_summary_query(table_name="master_egitim")
# df = client.query_df(query)
# row = df.iloc[0].to_dict()
# response = format_alerts_summary(row)
# ---
# get_promotion_history(client)
# get_similar_campaigns(client)
# ---
# response = get_forecast_calendar(
#     client=client,
#     table_name="master_egitim",
#     store_ids=["1001", "1002"],  # ⚠️ artık numeric ama string olarak
#     month=2,
#     year=2025
# )

# # 3) Output
# response
# ---
# response = get_inventory_kpis(
#     client=client,
#     table_name="master_egitim",
#     region_ids=["Marmara"],
#     store_ids=["1001"],
#     category_ids=["104"],
#     product_ids=["30000264"]
# )
# ---
# response = get_inventory_items(
#     client=client,
#     table_name="master_egitim",
#     region_ids=["Marmara"],
#     store_ids=["1001"],
#     category_ids=["104"],
#     status="Low Stock",
#     page=1,
#     limit=50,
#     sort_by="stockValue",
#     sort_order="desc"
# )

# response
# ---
# response = get_inventory_stock_trends(
#     client=client,
#     table_name="master_egitim",
#     region_ids=["Marmara"],
#     store_ids=["1001"],
#     category_ids=["104"],
#     product_ids=["30000264"],
#     days=30
# )

# response
# ---
# response = get_inventory_store_performance(
#     client=client,
#     table_name="master_egitim",
#     region_ids=["Marmara"],
#     category_ids=["104"],
# )

# response
# ---